In [34]:
import argparse
import os
import sys
import tempfile
from pathlib import Path

import cv2
import h5py
import numpy as np
from naoth.log import BoundingBox
from tqdm import tqdm



#tools_path = os.path.join(current_working_directory, "tools")
tools_path = "c:\\Users\\johan\\Documents\\GitHub\\naoth-deeplearning\\"
print(f"Adding {tools_path} to sys.path")
sys.path.append(tools_path)
print(sys.path)

from tools import (
    download_from_minio,
    get_labelstudio_client,
    get_minio_client,
    get_postgres_cursor,
    load_image_as_yuv422_y_only_pil,
)

Adding c:\Users\johan\Documents\GitHub\naoth-deeplearning\ to sys.path
['c:\\Users\\johan\\Documents\\GitHub\\naoth-deeplearning\\semantic_segmentation_lc', 'c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\johan\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\johan\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32', 'C:\\Users\\johan\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\johan\\AppData\\Roaming\\Python\\Python312\\site-packages\\Pythonwin', 'c:\\Program Files\\Python312\\Lib\\site-packages', 'c:\\Users\\johan\\Documents\\GitHub\\naoth-deeplearning\\tools', 'c:\\Users\\johan\\Documents\\GitHub\\naoth-deeplearning\\', 'c:\\Users\\johan\\Documents\\GitHub\\naoth-deeplearning\\']


ModuleNotFoundError: No module named 'label_studio_sdk'

In [15]:
def download_images_and_masks(camera, grid_size):
    sql_query = f"""SELECT ls_project_top FROM robot_logs WHERE ls_project_top = 494"""
    print(sql_query)
    pg_cur = get_postgres_cursor()
    pg_cur.execute(sql_query)
    rtn_val = pg_cur.fetchall()
    data = [x for x in rtn_val]

    mclient = get_minio_client()
    ls = get_labelstudio_client()

    for ls_prj, bucket_name in sorted(data):

        print(f"Working on project {ls_prj}")

        project = ls.get_project(ls_prj)
        tasks = project.get_labeled_tasks()

        # TODO download all files for a project
        # TODO move the files inside an h5 file in the current folder
        # TODO think about structure inside h5 file
        # TODO first draft I can make mask from bounding boxes like this: https://stackoverflow.com/questions/64195636/converting-bounding-box-regions-into-masks-and-saving-them-as-png-files

        download_folder = Path("./datasets") / camera
        download_folder_images = download_folder / "images"
        Path(download_folder).mkdir(exist_ok=True, parents=True)
        for task_output in tasks:
            # label part 1
            bbox_list_robot = list()
            bbox_list_penalty = list()
            bbox_list_ball = list()
            for anno in task_output["annotations"]:
                results = anno["result"]
                for result in results:
                    # ignore relations here
                    if result["type"] != "rectanglelabels":
                        continue
                    
                    actual_label = result["value"]["rectanglelabels"][0]

                    # x,y,width,height are all percentages within [0,100]
                    x, y, width, height = (
                        result["value"]["x"],
                        result["value"]["y"],
                        result["value"]["width"],
                        result["value"]["height"],
                    )
                    img_width = result["original_width"]
                    img_height = result["original_height"]
                    # FIXME int might not be the best rounding method here - but off by one pixel is also not that bad
                    x_px = int(x / 100 * img_width)
                    y_px = int(y / 100 * img_height)
                    width_px = int(width / 100 * img_width)
                    height_px = int(height / 100 * img_height)

                    if actual_label == "ball":
                        bbox_list_ball.append((y_px, height_px, x_px, width_px))
                    if actual_label == "penalty_mark":
                        bbox_list_penalty.append((y_px, height_px, x_px, width_px))
                    if actual_label == "nao":
                        bbox_list_robot.append((y_px, height_px, x_px, width_px))

            # TODO creating the masks makes it harder to calculate overlap with grid cells later so do it here and the output will then always be in the grid shape and not the image shape
            # TODO figure out how a better workflow that also includes the adjustments to the robot masks could work -> after robocup and with new labeltool
            if len(bbox_list_ball) > 0 or len(bbox_list_penalty) > 0 or len(bbox_list_robot) > 0:
                # image part
                image_file_name = task_output["storage_filename"]
                image_path = download_from_minio(
                    client=mclient,
                    bucket_name=bucket_name,
                    filename=image_file_name,
                    output_folder=download_folder_images,
                )

                # label part 2
                img = cv2.imread(image_path)
                img_height = img.shape[0]
                img_width = img.shape[1]

                grid_rows, grid_columns = (
                    grid_size  # FIXME better names its not actually height but num cols num rows or something like that
                )
                grid_cell_height = int(img_height / grid_rows)
                grid_cell_width = int(img_width / grid_columns)

                # use defined grid shape here
                mask = np.zeros((grid_rows, grid_columns, 3), dtype=np.float32)  # initialize mask

                for box in bbox_list_ball:
                    y_px, height_px, x_px, width_px = box
                    ball_bb = BoundingBox.from_xywh(x_px, y_px, width_px, height_px)
                    for y in range(grid_rows):
                        for x in range(grid_columns):
                            cell_x1 = x * grid_cell_width
                            cell_y1 = y * grid_cell_height
                            cell_x2 = x * grid_cell_width + grid_cell_width
                            cell_y2 = y * grid_cell_height + grid_cell_height
                            cell_bb = BoundingBox.from_coords(cell_x1, cell_y1, cell_x2, cell_y2)
                            intersection = cell_bb.intersection(ball_bb)
                            if not intersection is None:
                                value = intersection.area / cell_bb.area

                                mask[y, x, 0] = value * 255.0  # because png can only handle ints argh

                for box in bbox_list_penalty:
                    y_px, height_px, x_px, width_px = box
                    # TODO put this in an extra function
                    penalty_bb = BoundingBox.from_xywh(x_px, y_px, width_px, height_px)
                    for y in range(grid_rows):
                        for x in range(grid_columns):
                            cell_x1 = x * grid_cell_width
                            cell_y1 = y * grid_cell_height
                            cell_x2 = x * grid_cell_width + grid_cell_width
                            cell_y2 = y * grid_cell_height + grid_cell_height
                            cell_bb = BoundingBox.from_coords(cell_x1, cell_y1, cell_x2, cell_y2)
                            intersection = cell_bb.intersection(penalty_bb)
                            if not intersection is None:
                                value = intersection.area / cell_bb.area
                                mask[y, x, 1] = value * 255.0  # because png can only handle ints argh

                for box in bbox_list_robot:
                    y_px, height_px, x_px, width_px = box
                    # TODO put this in an extra function
                    robot_bb = BoundingBox.from_xywh(x_px, y_px, width_px, height_px)
                    for y in range(grid_rows):
                        for x in range(grid_columns):
                            cell_x1 = x * grid_cell_width
                            cell_y1 = y * grid_cell_height
                            cell_x2 = x * grid_cell_width + grid_cell_width
                            cell_y2 = y * grid_cell_height + grid_cell_height
                            cell_bb = BoundingBox.from_coords(cell_x1, cell_y1, cell_x2, cell_y2)
                            intersection = cell_bb.intersection(robot_bb)
                            if not intersection is None:
                                value = intersection.area / cell_bb.area
                                mask[y, x, 2] = value * 255.0  # because png can only handle ints argh

                # maybe use different output folders for different grid sizes?
                a = Path(download_folder) / "label"
                Path(a).mkdir(exist_ok=True, parents=True)
                mask_output_path = Path(download_folder) / "label" / str(bucket_name + "_" + image_file_name)
                cv2.imwrite(str(mask_output_path), mask)